### Import Libraries

In [9]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import cv2
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Conv2D, MaxPooling2D, Flatten, Dropout
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.callbacks import EarlyStopping

### Loading and Preprocessing data

In [10]:
# Define the paths to the dataset folders
data_path ='data'
non_vehicle_folder = os.path.join(data_path, 'non-vehicles')
vehicle_folder = os.path.join(data_path, 'vehicles')

# Initialize lists to hold image data and labels
images = []
labels = []

# Counters for labeling
car_count = 0
non_car_count = 0

# Helper function to process images from a given folder
def process_images_from_folder(folder_path, is_vehicle):
    global car_count, non_car_count
    for img_name in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_name)
        img = cv2.imread(img_path)
        if img is None:
            continue  # Skip if the image couldn't be read
        
        # Resize image to 224x224 pixels
        img = cv2.resize(img, (224, 224))
        images.append(img)
        
        # Assign labels based on whether it's a vehicle or non-vehicle
        if is_vehicle:
            label = 1  # Label for car
            car_count += 1
        else:
            label = 0  # Label for non-car
            non_car_count += 1
        
        labels.append(label)

# Process images from both folders
process_images_from_folder(vehicle_folder, is_vehicle=True)
process_images_from_folder(non_vehicle_folder, is_vehicle=False)

# Convert lists to numpy arrays
images = np.array(images)
labels = np.array(labels)

print("Images shape:", images.shape)
print("Labels:", labels)


Images shape: (17760, 224, 224, 3)
Labels: [1 1 1 ... 0 0 0]


### Normalizing and Splitting data

In [11]:
# Normalize the images
images = images / 255.0

# Convert labels to categorical
labels = to_categorical(labels, num_classes=2)

# Split the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

### Building the Model

CNN Model experimenting with the potential benefits by simply adding layers

In [12]:
# Initialize the model
model = Sequential()

# Add an Input layer as the first layer in the model
model.add(Input(shape=(224, 224, 3)))

# Add convolutional layers
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(128, (3, 3), activation='relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))

# Flatten the output
model.add(Flatten())

# Add dense layers
model.add(Dense(512, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Summary of the model
model.summary()


Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_3 (Conv2D)           (None, 222, 222, 32)      896       
                                                                 
 max_pooling2d_3 (MaxPoolin  (None, 111, 111, 32)      0         
 g2D)                                                            
                                                                 
 dropout_4 (Dropout)         (None, 111, 111, 32)      0         
                                                                 
 conv2d_4 (Conv2D)           (None, 109, 109, 64)      18496     
                                                                 
 max_pooling2d_4 (MaxPoolin  (None, 54, 54, 64)        0         
 g2D)                                                            
                                                                 
 dropout_5 (Dropout)         (None, 54, 54, 64)       

In [13]:
# Initialize the image data generator
datagen = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

# Fit the data generator on the training data
datagen.fit(X_train)

### Training the Model

In [14]:
# Callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)

# Train the model using the data generator
history = model.fit(datagen.flow(X_train, y_train, batch_size=128), 
                    epochs=50, 
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping])

Epoch 1/50
111/111 [==============================] - 99s 881ms/step - loss: 0.7736 - accuracy: 0.7140 - val_loss: 0.2495 - val_accuracy: 0.9065
Epoch 2/50
111/111 [==============================] - 96s 866ms/step - loss: 0.2393 - accuracy: 0.9123 - val_loss: 0.1559 - val_accuracy: 0.9510
Epoch 3/50
111/111 [==============================] - 96s 863ms/step - loss: 0.1507 - accuracy: 0.9439 - val_loss: 0.1392 - val_accuracy: 0.9409
Epoch 4/50
111/111 [==============================] - 95s 856ms/step - loss: 0.1203 - accuracy: 0.9554 - val_loss: 0.0814 - val_accuracy: 0.9803
Epoch 5/50
111/111 [==============================] - 95s 852ms/step - loss: 0.0966 - accuracy: 0.9654 - val_loss: 0.0659 - val_accuracy: 0.9825
Epoch 6/50
111/111 [==============================] - 96s 864ms/step - loss: 0.0884 - accuracy: 0.9688 - val_loss: 0.0630 - val_accuracy: 0.9828
Epoch 7/50
111/111 [==============================] - 97s 874ms/step - loss: 0.0747 - accuracy: 0.9751 - val_loss: 0.0570 - val_ac

### Evaluating the Model

In [15]:
# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {accuracy*100:.2f}%')

111/111 [==============================] - 2s 18ms/step - loss: 0.0139 - accuracy: 0.9958
Test Accuracy: 99.58%


### Making Predictions